# I.Import Library





In [25]:

%tensorflow_version 1.x
!pip install keras==2.2.5
!pip install pyvi

import numpy as np
from numpy import random
import os, pickle, re, keras, sklearn, string
from keras.callbacks import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pyvi import ViTokenizer, ViPosTagger
from keras.layers import *
from keras.models import Model
from keras import optimizers
import gensim, operator, json
import pandas as pd
from sklearn.metrics import *
import keras.backend as K
from keras.models import *
from keras import initializers, regularizers
from keras import optimizers
from keras.engine.topology import Layer
from keras import constraints

# II.Read Data

In [26]:
!wget https://thiaisotajppub.s3-ap-northeast-1.amazonaws.com/publicfiles/baomoi.model.bin

--2021-05-16 14:59:47--  https://thiaisotajppub.s3-ap-northeast-1.amazonaws.com/publicfiles/baomoi.model.bin
Resolving thiaisotajppub.s3-ap-northeast-1.amazonaws.com (thiaisotajppub.s3-ap-northeast-1.amazonaws.com)... 52.219.8.78
Connecting to thiaisotajppub.s3-ap-northeast-1.amazonaws.com (thiaisotajppub.s3-ap-northeast-1.amazonaws.com)|52.219.8.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 708212586 (675M) [application/macbinary]
Saving to: ‘baomoi.model.bin.2’

baomoi.model.bin.2  100%[===================>] 675.40M  19.8MB/s    in 36s     

2021-05-16 15:00:24 (18.9 MB/s) - ‘baomoi.model.bin.2’ saved [708212586/708212586]



In [27]:
import zipfile
with zipfile.ZipFile("UIT-VSMEC.zip","r") as zf:
    zf.extractall()
print(zf)

<zipfile.ZipFile [closed]>


In [28]:
path_train ='comments_Task1.csv'
path_valid ='data/valid_nor_811.csv'
path_test ='data/test_nor_811.csv'
path_stopword = 'data/stopwords.txt'

# III.Word2vec using baomoi.model.bin

*   Function reading pretrain word embedding library.
*   The word embedding pretrain has been trained in new news, 300-way news



In [29]:
path_embedding= 'baomoi.model.bin'

import io
from gensim.models import KeyedVectors
word_embedding = KeyedVectors.load_word2vec_format(path_embedding, binary=True)
EMBEDDING_DIM = word_embedding['yêu'].shape[0]
print("Embedding: ",EMBEDDING_DIM)
print(word_embedding['yêu'])


Embedding:  400
[-0.78774583 -0.22327825 -0.6274532  -2.7228408  -2.2186291   0.38002455
  3.8660462   0.9853684  -1.4683082  -1.7013292  -0.5839958  -0.14467287
  3.600142    3.381808   -0.02930526  3.0047843  -0.2006207  -1.0937127
  1.7360235   2.3691583  -0.71597415  3.319453    0.2824182  -3.0814204
  2.6810844  -0.810977    1.5186927  -2.10329     1.3271075  -1.3646411
 -0.11144319 -4.6505136  -1.7251624  -2.31126     1.583203   -0.8746506
 -2.6937015  -1.7733976   0.557898   -1.7562917   1.3282276  -0.3805479
 -1.3979301  -0.1536707  -1.1909302   1.3283668   0.22275637 -2.7959821
 -5.188217   -0.6404673   0.0164395   0.67177856 -1.4948794   0.21867418
 -1.4103376   0.99262404  2.2180524  -0.4881204   3.0988753  -0.31382522
  1.3226501   0.21269594 -1.6409203   1.7758838   2.3379912  -2.4666297
 -0.599687    0.551105   -1.3755493  -1.4293027  -2.6366289   0.40759587
 -0.77850854 -0.6169452  -0.84525913  0.02801617  2.1296268   0.13715844
 -1.1562283  -2.1226277  -0.1346792   0.88

# IV. Data Preprocessing

In [30]:
def tokenizer(text):
    token = ViTokenizer.tokenize(text)
    return token

In [31]:
def deleteIcon(text):
    text = text.lower()
    s = ''
    pattern = r"[a-zA-ZaăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ,._]"
    for char in text:
        if char !=' ':
            if len(re.findall(pattern, char)) != 0:
                s+=char
            elif char == '_':
                s+=char
        else:
            s+=char
    s = re.sub('\\s+',' ',s)
    return s.strip()

In [32]:
def clean_doc(doc):
    doc = tokenizer(doc)
    for punc in string.punctuation:
        if punc != "_":
            doc = doc.replace(punc,' ')
    doc = deleteIcon(doc) 
    doc = re.sub(r"[0-9]+", " num ", doc)
    doc = doc.lower()
    doc = re.sub('\\s+',' ',doc)
    return doc

In [33]:
# from underthesea import word_tokenize
def pre_process(questions):
    stop_words = stopwords.words("english")
    questions_stop = [[t for t in tokens if (t not in stop_words) and (3 < len(t.strip()) < 15)]
                      for tokens in questions_tokens]
    questions_stop = pd.Series(questions_stop)
    return questions_stop

# V.Train/Test data

In [34]:
train_data = pd.read_csv(path_train)
valid_data = pd.read_csv(path_valid)
test_data = pd.read_csv(path_test)

X_train = train_data["Sentence"].apply(lambda x : clean_doc(x))
y_train = train_data["Emotion"]

X_val = valid_data["Sentence"].apply(lambda x : clean_doc(x))
y_val = valid_data["Emotion"]

X_test = test_data["Sentence"].apply(lambda x : clean_doc(x))
y_test = test_data["Emotion"]

In [35]:
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))
print(len(X_test),len(y_test))


421 421
686 686
693 693


In [36]:
classes = ['Anger','Disgust','Enjoyment','Fear','Other','Sadness','Surprise']
def to_category_vector(label):
    vector = np.zeros(len(classes)).astype(np.float64)
    index = classes.index(label)
    vector[index] = 1.0
    return vector

In [37]:
y_train_encode = []
for label in y_train:
    y_train_encode.append(to_category_vector(label))

y_val_encode = []
for label in y_val:
    y_val_encode.append(to_category_vector(label))

print(classes)
print(y_train_encode[0])
print(y_train[0])

['Anger', 'Disgust', 'Enjoyment', 'Fear', 'Other', 'Sadness', 'Surprise']
[0. 0. 1. 0. 0. 0. 0.]
Enjoyment


### LSTM


*   All the words in the X_train set will form a dictionary
*   Each vector of the input word, it will turn into a vector with a fixed number of dimensions and each vocabulary will be replaced by its index in the dictionary
* Number of vector dimensions per input we will take the longest sentence which is the direction of the vector and the shorter arcs will automatically add the value 0 after

In [38]:
xLengths = [len(x.split(' ')) for x in X_train]
h = sorted(xLengths)  
maxLength =h[len(h)-1]
print("The longest sentence length value: ",maxLength)
input_tokenizer = Tokenizer(filters="",oov_token="UNK")
input_tokenizer.fit_on_texts(X_train)
input_vocab_size = len(input_tokenizer.word_index) + 1
word_index = input_tokenizer.word_index
print("input_vocab_size:",input_vocab_size)
X_train_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_train), maxlen=maxLength,padding="post"))

The longest sentence length value:  268
input_vocab_size: 1104


In [39]:
print("Input String : ", X_train[0])
print("Encode : ",X_train_encode[0])

X_val_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_val), maxlen=maxLength,padding="post"))

Input String :  dân_chơi xóm giàu vì bạn sang vì vợ ala_ela đây là rap_việt
Encode :  [105  89  38  10  17  33  10  34 189  25   4 132   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  

In [40]:
def generate_embedding(word_index, model_embedding,EMBEDDING_DIM):
    count6 = 0
    countNot6 = 0
    embedding_matrix = np.asarray([np.random.uniform(-0.01,0.01,EMBEDDING_DIM) for _ in range((len(word_index) + 1))])
    list_oov = []
    word_is_trained = []
    for word, i in word_index.items():
        try:
            embedding_vector = model_embedding[word]
            word_is_trained.append(word)
        except:
            continue
        if embedding_vector is not None:
            count6 +=1
            embedding_matrix[i] = embedding_vector
    
    print('Number of words in pre-train embedding: ' + str(count6))
    print('Number of words not in pre-train embedding: ' + str(countNot6))
    return embedding_matrix,word_is_trained

In [41]:
embedding_matrix,word_is_trained = generate_embedding(word_index,word_embedding,EMBEDDING_DIM)
print(word_is_trained)

Number of words in pre-train embedding: 987
Number of words not in pre-train embedding: 0
['hay', 'bài', 'là', 'nghe', 'rap', 'của', 'mck', 'có', 'vì', 'vẫn', 'em', 'nhất', 'thì', 'cũng', 'quá', 'bạn', 'luôn', 'ko', 'việt', 'kim', 'mình', 'anh', 'mà', 'đây', 'thang', 'gì', 'và', 'người', 'bắc', 'nhưng', 'k', 'sang', 'vợ', 'như', 'nào', 'sao', 'giàu', 'đỉnh', 'rồi', 'các', 'ai', 'hơn', 'thấy', 'để', 'ạ', 'làm', 'cho', 'đã', 'binz', 'được', 'về', 'karik', 'cái', 'này', 'nó', 'xem', 'con', 'nhiều', 'đến', 'thích', 'mấy', 'nhạc', 'đó', 'từ', 'tr', 'ấy', 'vào', 'đúng', 'cứ', 'đầu', 'view', 'bác', 'ricky', 'hát', 'chỉ', 'với', 'đc', 'tui', 'r', 'mọi', 'lại', 'cả', 'nón', 'tiền', 'xóm', 'bắc_kim_thang', 'khi', 'nổi', 'ông', 'biết', 'bị', 'à', 'không', 'giọng', 'đang', 'trên', 'triệu', 'thuộc', 'chất', 'thêm', 'dân_chơi', 'sau', 'đầu_tiên', 'nay', 'da', 'ráp', 'hết', 'phải', 'tất_cả', 'vl', 'nữa', 'xong', 'thiếu', 'vs', 'yêu', 'mới', 'trong', 'dùng', 'kiểu', 'x', 'rất', 'tôi', 'nói', 'hiểu', '

# VI.Attention Layer

In [42]:
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        a = K.exp(ait)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a

        return weighted_input

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[1], input_shape[2]

class Addition(Layer):
    def __init__(self, **kwargs):
        super(Addition, self).__init__(**kwargs)

    def build(self, input_shape):
        self.output_dim = input_shape[-1]
        super(Addition, self).build(input_shape)

    def call(self, x):
        return K.sum(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

## 1.Build mode LSTM

In [43]:
filter_nums = 256 
def build_model():
        inputs  = Input(shape=(maxLength, ), dtype='float64', name='inputs')    
        embedding_layer = Embedding(input_vocab_size,EMBEDDING_DIM,weights=[embedding_matrix], input_length=maxLength, trainable=True,name = 'word_emb')(inputs)
        embedding_layer = SpatialDropout1D(0.75)(embedding_layer)     
        lstm_feature1 = CuDNNLSTM(filter_nums, return_sequences=True)(embedding_layer)
        att1 = AttentionWithContext()(lstm_feature1)
        att1 = Addition()(att1)
        fc1 = Dropout(0.5)(Dense(256, name = 'dense_1')(att1))
        output1 = Dense(len(classes),name="output1", activation='softmax')(fc1)
        model = Model(inputs=inputs, outputs=output1)
        tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
        model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])   
        history = model.fit(X_train_encode, np.array(y_train_encode), validation_data = (X_val_encode,np.array(y_val_encode)) , batch_size=50, epochs=100,callbacks=[tensorBoardCallback])
        return model

model = build_model()

Train on 421 samples, validate on 686 samples
Epoch 1/100
421/421 [==============================] - 1s 2ms/step - loss: 1.6655 - acc: 0.5796 - val_loss: 1.8840 - val_acc: 0.3120
Epoch 2/100
421/421 [==============================] - 0s 1ms/step - loss: 1.3493 - acc: 0.6437 - val_loss: 2.3520 - val_acc: 0.3120
Epoch 3/100
421/421 [==============================] - 0s 1ms/step - loss: 1.1942 - acc: 0.6461 - val_loss: 2.1503 - val_acc: 0.3105
Epoch 4/100
421/421 [==============================] - 0s 1ms/step - loss: 1.1849 - acc: 0.6366 - val_loss: 1.9698 - val_acc: 0.2857
Epoch 5/100
421/421 [==============================] - 0s 1ms/step - loss: 1.1448 - acc: 0.6437 - val_loss: 2.0546 - val_acc: 0.3090
Epoch 6/100
421/421 [==============================] - 0s 1ms/step - loss: 1.1008 - acc: 0.6366 - val_loss: 2.0904 - val_acc: 0.3032
Epoch 7/100
421/421 [==============================] - 0s 1ms/step - loss: 1.0913 - acc: 0.6461 - val_loss: 2.0816 - val_acc: 0.2915
Epoch 8/100
421/421 [==

## 2.Predict the results

In [44]:
X_test_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences(X_test), maxlen=maxLength,padding="post"))
test_length = len(X_test_encode)

y_predict = []
predicted = model.predict(X_test_encode)
for predict in predicted:
    index2, value = max(enumerate(predict), key=operator.itemgetter(1))
    y_predict.append(classes[index2])
    
print(y_predict[0])

Enjoyment


In [45]:
print(y_predict)

['Enjoyment', 'Other', 'Disgust', 'Sadness', 'Enjoyment', 'Enjoyment', 'Other', 'Enjoyment', 'Sadness', 'Sadness', 'Disgust', 'Enjoyment', 'Disgust', 'Other', 'Enjoyment', 'Enjoyment', 'Other', 'Fear', 'Surprise', 'Enjoyment', 'Enjoyment', 'Surprise', 'Sadness', 'Other', 'Enjoyment', 'Other', 'Other', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Other', 'Other', 'Disgust', 'Enjoyment', 'Other', 'Other', 'Other', 'Other', 'Surprise', 'Other', 'Other', 'Other', 'Other', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Other', 'Other', 'Disgust', 'Fear', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Other', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Other', 'Enjoyment', 'Anger', 'Enjoyment', 'Other', 'Enjoyment', 'Other', 'Other', 'Sadness', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Fear', 'Sadness', 'Enjoyment', 'Enjoyment', 'Sadness', 'Enjoyment', 'Enjoyment', 'Fear', 'Enjoyment', 'Enjoyment', 'Enjoyment', 'Anger', 'Enjoyment', 'Surprise', 'Enjoyment', 'Other'

## 3.Report the performance metrics (Accuracy, F1-score...)

In [46]:
precision = precision_score(y_test, y_predict, average='weighted')
recall = recall_score(y_test, y_predict, average='weighted')
f1score = f1_score(y_test, y_predict, average='micro')
accuracy = accuracy_score(y_test, y_predict)

print("Result model LSTM + Attention layer")
print("Results of the models")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-Score: ", f1score)
print("Accuracy: ", accuracy)

print(classification_report(y_test,y_predict))

Result model LSTM + Attention layer
Results of the models
Precision:  0.27367567397178016
Recall:  0.27705627705627706
F1-Score:  0.27705627705627706
Accuracy:  0.27705627705627706
              precision    recall  f1-score   support

       Anger       0.06      0.03      0.04        40
     Disgust       0.29      0.16      0.20       132
   Enjoyment       0.36      0.52      0.42       193
        Fear       0.50      0.13      0.21        46
       Other       0.21      0.33      0.26       129
     Sadness       0.21      0.14      0.16       116
    Surprise       0.17      0.16      0.17        37

    accuracy                           0.28       693
   macro avg       0.26      0.21      0.21       693
weighted avg       0.27      0.28      0.26       693



# VII.Enter the demo program into 1 sentence

In [47]:
def demo(str):
  demo_pre = clean_doc(str)
  X_demo_encode = np.array(pad_sequences(input_tokenizer.texts_to_sequences([demo_pre]), maxlen=maxLength,padding="post"))
  predicted = model.predict(X_demo_encode)
  index2, value = max(enumerate(predicted[0]), key=operator.itemgetter(1))
  return classes[index2]

In [48]:
demo('4 : 05 ai biết info áo anh binz ko nhỉ')

'Surprise'